In [11]:
!pip install spacy

In [12]:
!python -m spacy info

2023-07-15 00:36:11.302417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.6.0                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.109+-x86_64-with-glibc2.31
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.5.0)        



In [11]:
 import spacy
 from spacy.tokens import DocBin
 from tqdm import tqdm

 nlp = spacy.blank("en") # load a new spacy model
 db = DocBin() # create a DocBin object

In [10]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/annotations.json')
TRAIN_DATA = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/annotations.json'

In [ ]:
TRAIN_DATA

In [33]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot["entities"]:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

  db.to_disk("/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/training_data.spacy")

100%|██████████| 222/222 [00:03<00:00, 66.34it/s]


In [36]:
! python -m spacy init config "/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/config.cfg" --lang en --pipeline ner --optimize efficiency

2023-07-15 01:40:03.738082: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [37]:
!python -m spacy train "/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/config.cfg" --output ./ --paths.train="/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/training_data.spacy" --paths.dev="/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/training_data.spacy"

2023-07-15 01:40:19.637349: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2023-07-15 01:40:23,152] [INFO] Set up nlp object from config
[2023-07-15 01:40:23,179] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-07-15 01:40:23,186] [INFO] Created vocabulary
[2023-07-15 01:40:23,186] [INFO] Finished initializing nlp object
[2023-07-15 01:40:23,772] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     31.70    0.77    0.58    1.14    0.01
  1     200

In [12]:
nlp_ner = spacy.load("model-best")

C:\Users\Geethan\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.6 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [13]:
doc = nlp_ner(''' I don't have muscle aches , sleepiness ''')

In [14]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [1]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Specify the folder path and output path
folder_path = '/content/drive/MyDrive/Colab Notebooks/Thyroid_symp/'  # Replace with your folder path
output_path = '/content/drive/MyDrive/Colab Notebooks/Thyroid_symp.zip'  # Replace with your desired output path

# Zip the folder
zip_folder(folder_path, output_path)